# Visualization

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import xgboost as xgb

from numpy import mean,sqrt,square
from sklearn.metrics import mean_squared_error
import pickle
import missingno as msno
import auxiliar_functions as af
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# read df full csv
df_features=pd.read_csv('../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#[variables_interes]#["2020":"2021"]
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

(859335, 403)

# Data test

In [5]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
# Ytest reg
Ytest=test["TPH"]
Xtest=test.drop(columns=["TPH"])

In [6]:
df1=Xtest.copy()

In [ ]:
df1["dif_TPH_HH"]=df1["HH TPH"]-df1["TPH"]
df1["dif_CC_HH"]=df1["HH charge cell"]-df1["charge cell"]
df1["dif_CC_LL"]=df1["charge cell"]-df1["LL charge cell"]

# Información de perdida de TPH en los proximos 5 minutos
for i in range(1,6):
    df1[f"loss of TPH_{i}"]=df1["loss of TPH"].shift(i)

df1["greater than HH CC"]=df1["dif_CC_HH"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df1[f"greater than HH CC_{i}"]=df1["greater than HH CC"].shift(i)

df1["less than LL CC"]=df1["dif_CC_LL"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df1[f"less than LL CC_{i}"]=df1["less than LL CC"].shift(i)

df1["greater than Rec HH CC"]=df1.apply(lambda x: 1 if x["HH charge cell"]<=x["charge cell"] else 0,axis=1)
for i in range(1,6):
    df1[f"greater than Rec HH CC_{i}"]=df1["greater than Rec HH CC"].shift(i)

df1["Rec HH CC less than HH CC"]=df1.apply(lambda x: 1 if x["HH charge cell"]<x["HH charge cell"] else 0,axis=1)
for i in range(1,6):
    df1[f"Rec HH CC less than HH CC_{i}"]=df1["Rec HH CC less than HH CC"].shift(i)

df1.dropna(inplace=True)
df1["loss of TPH"]=df1.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)
df1=af.loss_tph(df1,"loss of TPH")

df_full=df1.copy()

list_df=[]
for i in df_full[df_full["start loss tph"]==1].index:
    try:
        #i="2022-02-27 12:31:00"
        inicio_perdida=i
        test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
        fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
        inicio_evento=inicio_perdida-timedelta(minutes=5)
        data=df_full.copy()[inicio_evento:fin_perdida]
        #display(data)
        dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
        "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
        "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
        list_df.append(pd.DataFrame(dic_df,index=[1]))  
    except:
        #print(i)
        pass

df_tph_loss=pd.concat(list_df).reset_index(drop=True)


In [ ]:




# Perdida de TPH por superar recomendación [+]
evento1=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
    & (df_tph_loss["Supera recomendación del modelo"]==True)\
    & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
list_loss_tph=[]
for i in range(len(evento1)):
    data=df_full[evento1.iloc[i]["inicio perdida"]:evento1.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph1=sum(list_loss_tph)

# Perdida de TPH sin superar recomendación [-]
evento2=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==False)\
    & (df_tph_loss["Supera recomendación del modelo"]==False)\
    ].reset_index(drop=True)
list_loss_tph=[]
for i in range(len(evento2)):
    data=df_full[evento2.iloc[i]["inicio perdida"]:evento2.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph2=sum(list_loss_tph)

metric_final=tph1-tph2

return metric_final